In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.features import shapes
from rasterio.features import rasterize
from rasterio.transform import from_origin
import rasterio.mask
from rasterio.io import MemoryFile
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
uk_boundary_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk boundary.geojson"
lines_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_transmission_lines.geojson"
wind_speed_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/wind_speed.nc"
tif_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_dem_clipped_UK.tif"
target_crs = "EPSG:27700"

In [ ]:

with rasterio.open(tif_path) as src:
    dem = src.read(1).astype("float32")
    if src.nodata is not None:
        dem = np.where(dem == src.nodata, np.nan, dem)

    transform = src.transform
    crs = src.crs
    bounds = src.bounds
    profile = src.profile.copy()

dx = abs(transform.a)
dy = abs(transform.e)

dz_dy_pix, dz_dx_pix = np.gradient(dem)

is_degree_grid = (crs is not None) and bool(getattr(crs, "is_geographic", False))

if is_degree_grid:
    rows = np.arange(dem.shape[0], dtype=np.int32)
    lats = np.array(
        [rasterio.transform.xy(transform, r, 0, offset="center")[1] for r in rows],
        dtype=np.float32
    )

    dx_m = dx * (111320.0 * np.cos(np.deg2rad(lats)))
    dy_m = dy * 110574.0

    dz_dx = dz_dx_pix / dx_m[:, None]
    dz_dy = dz_dy_pix / dy_m
else:
    dz_dx = dz_dx_pix / dx
    dz_dy = dz_dy_pix / dy

slope_pct = np.sqrt(dz_dx**2 + dz_dy**2) * 100.0

print("Raster CRS:", crs)
print("Slope % (min / median / max):",
      np.nanmin(slope_pct), np.nanmedian(slope_pct), np.nanmax(slope_pct))

uk = gpd.read_file(uk_boundary_path)

if uk.crs != crs:
    try:
        uk = uk.to_crs(crs)
    except Exception as e:
        raise RuntimeError(
            "UK boundary CRS != raster CRS, and reprojection failed (PROJ issue).\n"
            "Reproject the UK boundary to the raster CRS in QGIS and save a new GeoJSON, then rerun.\n"
            f"Boundary CRS: {uk.crs}\nRaster CRS: {crs}\nError: {e}"
        )

uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
uk_union = uk.unary_union
geoms = [uk_union]

profile.update(count=1, dtype="float32")

with MemoryFile() as memfile:
    with memfile.open(**profile) as tmp:
        tmp.write(slope_pct.astype("float32"), 1)

        out_img, out_transform = rasterio.mask.mask(
            tmp, geoms, crop=True, filled=True, nodata=np.nan
        )

slope_uk = out_img[0]


h, w = slope_uk.shape
left, top = out_transform * (0, 0)
right, bottom = out_transform * (w, h)
extent = (left, right, bottom, top)


cmap = LinearSegmentedColormap.from_list("blue_yellow", ["blue", "yellow"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11))

im = ax.imshow(
    slope_uk,
    extent=extent,
    origin="upper",
    cmap=cmap,
    vmin=0,
    vmax=50
)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)
ax.set_axis_off()

cb = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cb.set_label("Slope (%)")
cb.set_ticks([0, 10, 20, 30, 40, 50])

plt.tight_layout()
plt.show()

In [ ]:

slope_mask = np.where(np.isfinite(slope_uk), (slope_uk > 15).astype(np.uint8), np.nan)
slope_mask = np.ma.masked_invalid(slope_mask)

cmap = ListedColormap(["green", "purple"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11))

ax.imshow(slope_mask, extent=extent, origin="upper", cmap=cmap, vmin=0, vmax=1)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.legend(
    handles=[
        Patch(facecolor="green", edgecolor="none", label="slope ≤ 15%"),
        Patch(facecolor="purple", edgecolor="none", label="slope > 15%"),
    ],
    loc="upper left",
    frameon=True
)

ax.set_axis_off()
plt.tight_layout()
plt.show()